In [1]:
import pandas as pd
import json
import sqlite3
import xml.etree.ElementTree as ET
from datetime import time, datetime, timedelta
import os
import requests

# [ETL] Extract

In [7]:
import pandas as pd
import json
import sqlite3
import xml.etree.ElementTree as ET
from functools import wraps

class Extract:
    def __init__(self, data_dir):
        self.data_dir = data_dir

    def handle_errors(f):
        """Error handling decorator for extraction methods."""
        @wraps(f)
        def decorated_function(*args, **kwargs):
            try:
                return f(*args, **kwargs)
            except FileNotFoundError as e:
                print(f"Error: File not found - {e}")
            except pd.errors.EmptyDataError:
                print(f"Error: The file {args[1]} is empty.")
            except pd.errors.ParserError as e:
                print(f"Error: CSV file {args[1]} is malformed - {e}")
            except json.JSONDecodeError as e:
                print(f"Error: Malformed JSON file {args[1]} - {e}")
            except sqlite3.DatabaseError as e:
                print(f"Error: Database issue with file {args[1]} - {e}")
            except ET.ParseError as e:
                print(f"Error: Malformed XML file {args[1]} - {e}")
            except UnicodeDecodeError as e:
                print(f"Error: Encoding issue in file {args[1]} - {e}")
            except KeyError as e:
                print(f"Error: Missing expected column or key - {e}")
            except Exception as e:
                print(f"An unexpected error occurred: {e}")
        return decorated_function

    @handle_errors
    def extract_csv(self, file_name):
        """Extracts data from a CSV file and removes any incomplete records."""
        file_path = self.data_dir + file_name
        df = pd.read_csv(file_path)
        # Drop rows with any empty or null values
        return df.dropna()

    @handle_errors
    def extract_json(self, file_name):
        """Extracts data from a JSON file and flattens nested structures."""
        file_path = self.data_dir + file_name
        with open(file_path, 'r') as f:
            data = json.load(f)

            # Prepare a list to hold the flattened data
            flattened_data = []

            # Loop through each building and its rooms
            for building, rooms in data.items():
                for room in rooms:
                    # Add a new dictionary with building name and room details
                    flattened_data.append({
                        'rid': room['id'],
                        'building': building,
                        'number': room['number'],
                        'capacity': room['capacity']
                    })

            # Create a DataFrame from the flattened data
            df = pd.DataFrame(flattened_data)
            # Drop rows with any empty or null values
            return df.dropna()

    @handle_errors
    def extract_db(self, file_name):
        """Extracts data from a SQLite database and removes any incomplete records."""
        file_path = self.data_dir + file_name
        conn = sqlite3.connect(file_path)
        query = "SELECT * FROM requisites"
        df = pd.read_sql_query(query, conn)
        conn.close()
        # Drop rows with any empty or null values
        return df.dropna()

    @handle_errors
    def extract_xml(self, file_name):
        """Extracts data from an XML file and removes incomplete records."""
        file_path = self.data_dir + file_name
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Wrap the content in a root element to make it valid XML
        wrapped_content = f"<AllCourses>{content}</AllCourses>"

        # Parse the wrapped XML
        root = ET.fromstring(wrapped_content)

        courses = []
        for course in root.findall('Courses'):
            # Extract data for each course
            class_info = {
                'classid': course.find('classid').text if course.find('classid') is not None else None,
                'cred': course.find('cred').text if course.find('cred') is not None else None,
                'description': course.find('description').text if course.find('description') is not None else None,
                'syllabus': course.find('syllabus').text if course.find('syllabus') is not None else None,
                'term': course.find('term').text if course.find('term') is not None else None,
                'years': course.find('years').text if course.find('years') is not None else None,
                'classes': {
                    'code': course.find('classes/code').text if course.find('classes/code') is not None else None,
                    'name': course.find('classes/name').text if course.find('classes/name') is not None else None,
                }
            }

            # Check if the class_info has any None values, only append full records
            if all(class_info.values()) and all(class_info['classes'].values()):
                courses.append(class_info)

        # Convert to DataFrame and drop any empty records
        df_courses = pd.DataFrame(courses)
        return df_courses.dropna()

    def extract_all(self):
        """Extracts data from all sources (CSV, JSON, SQLite, XML) and returns DataFrames."""
        courses_df = self.extract_xml('courses.xml')
        meeting_df = self.extract_csv('meeting.csv')
        requisites_df = self.extract_db('requisites.db')
        rooms_df = self.extract_json('rooms.json')
        sections_df = self.extract_csv('sections.csv')

        print("[ETL] Extraction complete.")

        return courses_df, meeting_df, requisites_df, rooms_df, sections_df


In [8]:
# Define the directory where the data files are stored
data_dir = "/Users/romerocruzsa/Downloads/rumad2.0_data/"

# Create an instance of the Extract class
extractor = Extract(data_dir)

# Extract all the data and store them in variables
courses_df, meeting_df, requisites_df, rooms_df, sections_df = extractor.extract_all()


[ETL] Extraction complete.


In [9]:
display(courses_df.head(10), courses_df.info())
display(meeting_df.head(10), meeting_df.info())
display(requisites_df.head(10), requisites_df.info())
display(rooms_df.head(10), rooms_df.info())
display(sections_df.head(10), sections_df.info())

# Before cleaning: 36 courses, 20 meetings, 59 requisites, 25 rooms, and 1333 sections
# After cleaning: 36 courses, 16 meetings, 59 requisites, 25 rooms, and 78 sections

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   classid      36 non-null     object
 1   cred         36 non-null     object
 2   description  36 non-null     object
 3   syllabus     36 non-null     object
 4   term         36 non-null     object
 5   years        36 non-null     object
 6   classes      36 non-null     object
dtypes: object(7)
memory usage: 2.1+ KB


,classid,cred,description,syllabus,term,years,classes
0,0002,4,Introduction to Computer Programming I,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '3015', 'name': 'CIIC'}"
1,0003,3,Foundations of Computing,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '3075', 'name': 'CIIC'}"
2,0004,3,Computer Architecture I,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '3081', 'name': 'CIIC'}"
3,0005,4,Advanced Programming,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4010', 'name': 'CIIC'}"
4,0006,4,Data Structures,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4020', 'name': 'CIIC'}"
5,0007,3,Analysis and Design of Algorithms,https://www.uprm.edu/cse/wp-content/uploads/si...,Second Semester,Every Year,"{'code': '4025', 'name': 'CIIC'}"
6,0008,3,Programming Languages,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '4030', 'name': 'CIIC'}"
7,0009,4,Operating Systems,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4050', 'name': 'CIIC'}"
8,0010,3,Senior Design Project I,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '4151', 'name': 'CIIC'}"
9,0011,3,Database Systems,https://www.uprm.edu/cse/wp-content/uploads/si...,Second Semester,Every Year,"{'code': '4060', 'name': 'CIIC'}"


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   mid     20 non-null     int64 
 1   ccode   20 non-null     int64 
 2   start   20 non-null     object
 3   end     20 non-null     object
 4   day     20 non-null     object
dtypes: int64(2), object(3)
memory usage: 932.0+ bytes


,mid,ccode,start,end,day
0,1,16,07:30:00,08:45:00,MJ
1,2,26,09:00:00,10:15:00,MJ
2,3,36,10:30:00,11:45:00,MJ
3,4,46,12:00:00,13:15:00,MJ
4,5,56,13:30:00,14:45:00,MJ
5,6,66,15:00:00,16:15:00,MJ
6,7,76,16:30:00,17:45:00,MJ
7,8,86,18:00:00,19:15:00,MJ
8,9,96,19:30:00,20:45:00,MJ
9,10,106,21:00:00,22:15:00,MJ


...

,rid,building,number,capacity
0,1,Stefani,113,120
1,2,Stefani,114,25
2,3,Stefani,121,30
3,4,Stefani,215,30
4,5,Stefani,226,40
5,6,Stefani,207,30
6,7,Stefani,214,30
7,8,Stefani,322,30
8,9,Stefani,317,28
9,10,Stefani,330,30


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1333 entries, 0 to 1332
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sid         1333 non-null   int64 
 1   room_id     1333 non-null   int64 
 2   meeting_id  1333 non-null   int64 
 3   class_id    1333 non-null   int64 
 4   semester    1333 non-null   object
 5   year        1333 non-null   int64 
 6   capacity    1333 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 73.0+ KB


,sid,room_id,meeting_id,class_id,semester,year,capacity
0,0,22,12,0,Spring,2025,22
1,1,15,23,12,Spring,2025,17
2,2,4,12,17,Spring,2025,42
3,3,9,9,8,Fall,2023,30
4,4,10,4,40,Spring,2022,15
5,5,11,23,36,Fall,2021,16
6,6,19,25,21,V2,2021,40
7,7,24,12,40,Spring,2020,48
8,8,29,2,39,V2,2025,45
9,9,24,4,26,Fall,2025,25


None

In [10]:
# 1. The classes data starts with id 2.
courses_df['classid'] = courses_df['classid'].astype(int)
for i, courses in courses_df.iterrows():
    if courses['classid'] < 2:
        courses_df = courses_df.drop(i)
display(courses_df)

,classid,cred,description,syllabus,term,years,classes
0,2,4,Introduction to Computer Programming I,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '3015', 'name': 'CIIC'}"
1,3,3,Foundations of Computing,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '3075', 'name': 'CIIC'}"
2,4,3,Computer Architecture I,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '3081', 'name': 'CIIC'}"
3,5,4,Advanced Programming,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4010', 'name': 'CIIC'}"
4,6,4,Data Structures,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4020', 'name': 'CIIC'}"
5,7,3,Analysis and Design of Algorithms,https://www.uprm.edu/cse/wp-content/uploads/si...,Second Semester,Every Year,"{'code': '4025', 'name': 'CIIC'}"
6,8,3,Programming Languages,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '4030', 'name': 'CIIC'}"
7,9,4,Operating Systems,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4050', 'name': 'CIIC'}"
8,10,3,Senior Design Project I,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '4151', 'name': 'CIIC'}"
9,11,3,Database Systems,https://www.uprm.edu/cse/wp-content/uploads/si...,Second Semester,Every Year,"{'code': '4060', 'name': 'CIIC'}"


In [11]:
# Merge sections with meetings to access start and end times
schedules_df = sections_df.merge(meeting_df, left_on='meeting_id', right_on='mid')

# Convert 'start' and 'end' columns to datetime for easier comparison
schedules_df['start'] = pd.to_datetime(schedules_df['start'], format='%H:%M:%S').dt.time
schedules_df['end'] = pd.to_datetime(schedules_df['end'], format='%H:%M:%S').dt.time

# Create a set to keep track of the sids to drop
sids_to_drop = set()

# 2. Check for two sections in the same room at the same time
for (room_id, day), group in schedules_df.groupby(['room_id', 'day']):
    for i, row1 in group.iterrows():
        for j, row2 in group.iterrows():
            if i >= j:  # Avoid duplicate checks and comparing the same section
                continue
            # Check if time intervals overlap
            if (row1['start'] < row2['end']) and (row2['start'] < row1['end']):
                # Drop the record with the highest sid
                if row1['sid'] > row2['sid']:
                    sids_to_drop.add(row1['sid'])
                else:
                    sids_to_drop.add(row2['sid'])

# 3. Check for sections taught at overlapping times for the same class
for class_id, group in schedules_df.groupby(['class_id']):
    for i, row1 in group.iterrows():
        for j, row2 in group.iterrows():
            if i >= j:  # Avoid duplicate checks and comparing the same section
                continue
            # Check if sections are taught at overlapping times on the same day
            if (row1['start'] < row2['end']) and (row2['start'] < row1['end']) and row1['day'] == row2['day']:
                # Drop the record with the highest sid
                if row1['sid'] > row2['sid']:
                    sids_to_drop.add(row1['sid'])
                else:
                    sids_to_drop.add(row2['sid'])

# Remove conflicting sections from the original sections_df
sections_df = sections_df[~sections_df['sid'].isin(sids_to_drop)].reset_index(drop=True)

# Display the remaining sections after conflicts are resolved
display(sections_df)

,sid,room_id,meeting_id,class_id,semester,year,capacity
0,0,22,12,0,Spring,2025,22
1,1,15,23,12,Spring,2025,17
2,2,4,12,17,Spring,2025,42
3,3,9,9,8,Fall,2023,30
4,4,10,4,40,Spring,2022,15
...,...,...,...,...,...,...,...
644,1310,7,21,32,V1,2025,57
645,1324,15,24,34,Spring,2019,33
646,1327,27,22,8,Spring,2021,40
647,1330,7,0,29,Spring,2018,31


In [12]:
# 4. Meeting ‘MJ’ are from 7:30 a.m.-10:15 a.m. and 12:30 p.m.-7:45 p.m.
# Convert 'start' and 'end' columns to time objects
meeting_df['time_start'] = pd.to_datetime(meeting_df['start']).dt.time
meeting_df['time_end'] = pd.to_datetime(meeting_df['end']).dt.time

# Define allowed time ranges
morning_start = time(7, 30)
morning_end = time(10, 15)
afternoon_start = time(12, 30)
afternoon_end = time(19, 45)

# Step 1: Remove invalid meetings outside allowed times
invalid_meetings = meeting_df[
    (meeting_df['day'] == 'MJ') & (
        (meeting_df['time_start'] < morning_start) | (meeting_df['time_end'] > morning_end) & 
        (meeting_df['time_start'] < afternoon_start) | (meeting_df['time_end'] > afternoon_end)
    )
]

# Drop invalid meetings
meeting_df = meeting_df.drop(invalid_meetings.index)

# Step 2: Adjust overlapping meetings
adjusted_meetings = []
for _, row in meeting_df.iterrows():
    new_start = row['time_start']
    new_end = row['time_end']

    # Convert time to datetime for easier manipulation
    start_dt = pd.to_datetime(str(new_start))
    end_dt = pd.to_datetime(str(new_end))

    # Check for overlaps with blocked times
    if row['day'] == 'MJ':
        if new_end > morning_end:  # If it overlaps with the morning block
            # Move to after the afternoon block
            new_start = (pd.to_datetime(str(afternoon_end)) + timedelta(minutes=15)).time()
            # Calculate the new end time based on the duration of the original meeting
            duration = end_dt - start_dt
            new_end = (pd.to_datetime(str(new_start)) + duration).time()
        
    # Update the row in place
    meeting_df.loc[meeting_df['mid'] == row['mid'], 'start'] = new_start.strftime("%H:%M:%S")
    meeting_df.loc[meeting_df['mid'] == row['mid'], 'end'] = new_end.strftime("%H:%M:%S")

# Drop time columns as they are no longer needed
meeting_df.drop(columns=['time_start', 'time_end'], inplace=True)


display(meeting_df)

/var/folders/pd/pytqjfv51bddy0szx15zr3_c0000gn/T/ipykernel_78413/463880103.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  meeting_df['time_start'] = pd.to_datetime(meeting_df['start']).dt.time
/var/folders/pd/pytqjfv51bddy0szx15zr3_c0000gn/T/ipykernel_78413/463880103.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  meeting_df['time_end'] = pd.to_datetime(meeting_df['end']).dt.time


,mid,ccode,start,end,day
0,1,16,07:30:00,08:45:00,MJ
1,2,26,09:00:00,10:15:00,MJ
4,5,56,20:00:00,21:15:00,MJ
5,6,66,20:00:00,21:15:00,MJ
6,7,76,20:00:00,21:15:00,MJ
7,8,86,20:00:00,21:15:00,MJ
10,11,10,07:30:00,08:20:00,LWV
11,12,20,08:30:00,09:20:00,LWV
12,13,30,09:30:00,10:20:00,LWV
13,14,40,10:30:00,11:20:00,LWV


In [13]:
# 6. ‘LMV’ meetings have a duration of 50 minutes; ‘MJ’ meetings have a duration of 75 minutes.
# Convert 'start' and 'end' columns to datetime for duration calculation
meeting_df['minute_start'] = pd.to_datetime(meeting_df['start'], format='%H:%M:%S')
meeting_df['minute_end'] = pd.to_datetime(meeting_df['end'], format='%H:%M:%S')

# Define expected durations for each type of meeting
expected_duration = {
    'LMV': 50.0,
    'MJ': 75.0
}

# Calculate duration in minutes
meeting_df['duration'] = (meeting_df['minute_end'] - meeting_df['minute_start']).dt.total_seconds() / 60

# Find conflicts where meetings do not comply with the required durations
duration_conflicts = meeting_df[
    meeting_df.apply(lambda row: expected_duration.get(row['day']) != row['duration'], axis=1)
]
# Drop the temporary columns
meeting_df.drop(columns=['minute_start', 'minute_end', 'duration'], inplace=True)


display(meeting_df)

,mid,ccode,start,end,day
0,1,16,07:30:00,08:45:00,MJ
1,2,26,09:00:00,10:15:00,MJ
4,5,56,20:00:00,21:15:00,MJ
5,6,66,20:00:00,21:15:00,MJ
6,7,76,20:00:00,21:15:00,MJ
7,8,86,20:00:00,21:15:00,MJ
10,11,10,07:30:00,08:20:00,LWV
11,12,20,08:30:00,09:20:00,LWV
12,13,30,09:30:00,10:20:00,LWV
13,14,40,10:30:00,11:20:00,LWV


In [14]:
# 7. Sections cannot be in overcapacity
# Merge sections with rooms to access capacity
sections_availability = sections_df.merge(rooms_df, left_on='room_id', right_on='rid', how='left')

# Identify sections with overcapacity
overcapacity_sections = sections_availability[sections_availability['capacity_x'] > sections_availability['capacity_y']]

# Get the sids of sections to drop
sids_to_drop = overcapacity_sections['sid'].tolist()

# Drop the overcapacity sections from the original sections_df
sections_df = sections_df[~sections_df['sid'].isin(sids_to_drop)]

display(sections_df)

,sid,room_id,meeting_id,class_id,semester,year,capacity
0,0,22,12,0,Spring,2025,22
1,1,15,23,12,Spring,2025,17
4,4,10,4,40,Spring,2022,15
5,5,11,23,36,Fall,2021,16
6,6,19,25,21,V2,2021,40
...,...,...,...,...,...,...,...
641,1293,22,25,18,Fall,2019,20
642,1298,1,0,34,Spring,2023,69
643,1300,23,9,0,Spring,2024,18
646,1327,27,22,8,Spring,2021,40


In [15]:
# 9. Sections must be taught in a valid classroom and meeting, and the class must exist
valid_enrollment_df = sections_df.merge(courses_df, left_on="class_id", right_on="classid")
valid_enrollment_df = valid_enrollment_df.merge(meeting_df, left_on="meeting_id", right_on="mid")
valid_enrollment_df = valid_enrollment_df.merge(rooms_df, left_on="room_id", right_on="rid")

# Get the list of valid `sid` values from valid_enrollment_df
valid_sids = valid_enrollment_df['sid'].values

# Filter sections_df to keep only rows with `sid` in valid_sids
sections_df = sections_df[sections_df['sid'].isin(valid_sids)]

display(sections_df)

,sid,room_id,meeting_id,class_id,semester,year,capacity
12,12,14,19,27,Fall,2022,28
13,13,22,1,21,Spring,2022,35
15,15,11,6,15,V2,2020,30
16,16,2,2,35,Spring,2019,18
17,17,1,17,3,Spring,2021,16
...,...,...,...,...,...,...,...
532,925,19,16,28,Spring,2018,64
540,944,21,8,16,Fall,2018,16
560,995,19,17,11,Fall,2020,37
562,1003,25,11,32,Fall,2022,36


In [16]:
# 10. If in courses_df, classes['name'] == "Authorization from the Director of the Department", keep the record.

In [17]:
# 11. Download course syllabi (assuming you have the URLs stored)
    # Create a directory to save syllabi if it doesn't exist
syllabus_dir = "/Users/romerocruzsa/Workspace/Projects/Classes/CIIC4060/rumad-v2-app-no-pensamos-repetir-npr/syllabuses"

    # Function to download syllabi
def download_syllabus(url, filename):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad responses
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {filename}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {url}: {e}")

    # Download each syllabus
for index, row in courses_df.iterrows():
        syllabus_url = row['syllabus']
        syllabus_filename = f'{syllabus_dir}/{row["classes"]['name']}-{row["classes"]['code']}-{row["description"]}.pdf'
        download_syllabus(syllabus_url, syllabus_filename)

Downloaded: /Users/romerocruzsa/Workspace/Projects/Classes/CIIC4060/rumad-v2-app-no-pensamos-repetir-npr/syllabuses/CIIC-3015-Introduction to Computer Programming I.pdf
Downloaded: /Users/romerocruzsa/Workspace/Projects/Classes/CIIC4060/rumad-v2-app-no-pensamos-repetir-npr/syllabuses/CIIC-3075-Foundations of Computing.pdf
Downloaded: /Users/romerocruzsa/Workspace/Projects/Classes/CIIC4060/rumad-v2-app-no-pensamos-repetir-npr/syllabuses/CIIC-3081-Computer Architecture I.pdf
Downloaded: /Users/romerocruzsa/Workspace/Projects/Classes/CIIC4060/rumad-v2-app-no-pensamos-repetir-npr/syllabuses/CIIC-4010-Advanced Programming.pdf
Downloaded: /Users/romerocruzsa/Workspace/Projects/Classes/CIIC4060/rumad-v2-app-no-pensamos-repetir-npr/syllabuses/CIIC-4020-Data Structures.pdf
Downloaded: /Users/romerocruzsa/Workspace/Projects/Classes/CIIC4060/rumad-v2-app-no-pensamos-repetir-npr/syllabuses/CIIC-4025-Analysis and Design of Algorithms.pdf
Downloaded: /Users/romerocruzsa/Workspace/Projects/Classes/CI

In [18]:
display(courses_df.head(10), courses_df.info())
display(meeting_df.head(10), meeting_df.info())
display(requisites_df.head(10), requisites_df.info())
display(rooms_df.head(10), rooms_df.info())
display(sections_df.head(10), sections_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   classid      36 non-null     int64 
 1   cred         36 non-null     object
 2   description  36 non-null     object
 3   syllabus     36 non-null     object
 4   term         36 non-null     object
 5   years        36 non-null     object
 6   classes      36 non-null     object
dtypes: int64(1), object(6)
memory usage: 2.1+ KB


,classid,cred,description,syllabus,term,years,classes
0,2,4,Introduction to Computer Programming I,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '3015', 'name': 'CIIC'}"
1,3,3,Foundations of Computing,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '3075', 'name': 'CIIC'}"
2,4,3,Computer Architecture I,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '3081', 'name': 'CIIC'}"
3,5,4,Advanced Programming,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4010', 'name': 'CIIC'}"
4,6,4,Data Structures,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4020', 'name': 'CIIC'}"
5,7,3,Analysis and Design of Algorithms,https://www.uprm.edu/cse/wp-content/uploads/si...,Second Semester,Every Year,"{'code': '4025', 'name': 'CIIC'}"
6,8,3,Programming Languages,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '4030', 'name': 'CIIC'}"
7,9,4,Operating Systems,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4050', 'name': 'CIIC'}"
8,10,3,Senior Design Project I,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '4151', 'name': 'CIIC'}"
9,11,3,Database Systems,https://www.uprm.edu/cse/wp-content/uploads/si...,Second Semester,Every Year,"{'code': '4060', 'name': 'CIIC'}"


None

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 0 to 19
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   mid     16 non-null     int64 
 1   ccode   16 non-null     int64 
 2   start   16 non-null     object
 3   end     16 non-null     object
 4   day     16 non-null     object
dtypes: int64(2), object(3)
memory usage: 768.0+ bytes


,mid,ccode,start,end,day
0,1,16,07:30:00,08:45:00,MJ
1,2,26,09:00:00,10:15:00,MJ
4,5,56,20:00:00,21:15:00,MJ
5,6,66,20:00:00,21:15:00,MJ
6,7,76,20:00:00,21:15:00,MJ
7,8,86,20:00:00,21:15:00,MJ
10,11,10,07:30:00,08:20:00,LWV
11,12,20,08:30:00,09:20:00,LWV
12,13,30,09:30:00,10:20:00,LWV
13,14,40,10:30:00,11:20:00,LWV


...

,rid,building,number,capacity
0,1,Stefani,113,120
1,2,Stefani,114,25
2,3,Stefani,121,30
3,4,Stefani,215,30
4,5,Stefani,226,40
5,6,Stefani,207,30
6,7,Stefani,214,30
7,8,Stefani,322,30
8,9,Stefani,317,28
9,10,Stefani,330,30


None

<class 'pandas.core.frame.DataFrame'>
Index: 78 entries, 12 to 563
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sid         78 non-null     int64 
 1   room_id     78 non-null     int64 
 2   meeting_id  78 non-null     int64 
 3   class_id    78 non-null     int64 
 4   semester    78 non-null     object
 5   year        78 non-null     int64 
 6   capacity    78 non-null     int64 
dtypes: int64(6), object(1)
memory usage: 4.9+ KB


,sid,room_id,meeting_id,class_id,semester,year,capacity
12,12,14,19,27,Fall,2022,28
13,13,22,1,21,Spring,2022,35
15,15,11,6,15,V2,2020,30
16,16,2,2,35,Spring,2019,18
17,17,1,17,3,Spring,2021,16
18,18,2,14,4,Fall,2020,21
34,35,18,12,2,Spring,2018,22
39,40,8,18,10,Fall,2025,24
46,47,14,16,13,Fall,2017,28
48,49,25,20,15,Spring,2023,22


None

In [19]:
import pandas as pd
import os
from datetime import time, timedelta, datetime
import requests

class Transform:
    def __init__(self, courses_df, meetings_df, requisites_df, rooms_df, sections_df):
        self.courses_df = courses_df
        self.meetings_df = meetings_df
        self.requisites_df = requisites_df
        self.sections_df = sections_df
        self.rooms_df = rooms_df

    def clean_courses(self):
        """Ensure course IDs start from 2."""
        self.courses_df['classid'] = self.courses_df['classid'].astype(int)
        self.courses_df = self.courses_df[self.courses_df['classid'] >= 2].reset_index(drop=True)

    def resolve_section_conflicts(self):
        """Resolve conflicts where sections overlap in the same room or the same course."""
        # Merge sections with meetings to have access to start, end times, and date info.
        schedules_df = self.sections_df.merge(self.meetings_df, left_on='meeting_id', right_on='mid')
        schedules_df['start'] = pd.to_datetime(schedules_df['start'], format='%H:%M:%S').dt.time
        schedules_df['end'] = pd.to_datetime(schedules_df['end'], format='%H:%M:%S').dt.time

        sids_to_drop = set()
        
        # Check for sections that share the same `sid`.
        duplicate_sids = schedules_df[schedules_df.duplicated('sid', keep=False)]
        sids_to_drop.update(duplicate_sids['sid'].tolist())

        # Check for conflicts in the same room and day, but respect different semesters/years.
        for (room_id, day), group in schedules_df.groupby(['room_id', 'day']):
            # Sort by start time to make comparison easier.
            sorted_group = group.sort_values(by=['start', 'end'])
            
            for i, row1 in sorted_group.iterrows():
                for j, row2 in sorted_group.iterrows():
                    # Avoid duplicate checks and comparisons with itself.
                    if i >= j:
                        continue

                    # Check for overlapping times.
                    if (row1['start'] < row2['end']) and (row2['start'] < row1['end']):
                        # Only remove if the year and semester are the same.
                        if row1['year'] == row2['year'] and row1['semester'] == row2['semester']:
                            sids_to_drop.add(max(row1['sid'], row2['sid']))

        # Filter out the sections that are marked for deletion.
        self.sections_df = self.sections_df[~self.sections_df['sid'].isin(sids_to_drop)].reset_index(drop=True)

    def filter_meetings(self):
        """Filter out invalid meetings and adjust those that go beyond the valid time ranges."""
        self.meetings_df['time_start'] = pd.to_datetime(self.meetings_df['start']).dt.time
        self.meetings_df['time_end'] = pd.to_datetime(self.meetings_df['end']).dt.time

        morning_start = time(7, 30)
        morning_end = time(10, 15)
        afternoon_start = time(12, 30)
        afternoon_end = time(19, 45)

        invalid_meetings = self.meetings_df[
            (self.meetings_df['day'] == 'MJ') & (
                (self.meetings_df['time_start'] < morning_start) | 
                (self.meetings_df['time_end'] > morning_end) & 
                (self.meetings_df['time_start'] < afternoon_start) | 
                (self.meetings_df['time_end'] > afternoon_end)
            )
        ]
        self.meetings_df.drop(invalid_meetings.index, inplace=True)

        for _, row in self.meetings_df.iterrows():
            start_dt = pd.to_datetime(str(row['time_start']))
            end_dt = pd.to_datetime(str(row['time_end']))
            if row['day'] == 'MJ' and end_dt.time() > morning_end:
                new_start = (pd.to_datetime(str(afternoon_end)) + timedelta(minutes=15)).time()
                duration = end_dt - start_dt
                new_end = (pd.to_datetime(str(new_start)) + duration).time()
                self.meetings_df.loc[self.meetings_df['mid'] == row['mid'], ['start', 'end']] = \
                    [new_start.strftime("%H:%M:%S"), new_end.strftime("%H:%M:%S")]

        self.meetings_df.drop(columns=['time_start', 'time_end'], inplace=True)

    def validate_meeting_durations(self):
        """Validate that 'LMV' meetings are 50 minutes and 'MJ' meetings are 75 minutes."""
        self.meetings_df['minute_start'] = pd.to_datetime(self.meetings_df['start'], format='%H:%M:%S')
        self.meetings_df['minute_end'] = pd.to_datetime(self.meetings_df['end'], format='%H:%M:%S')
        self.meetings_df['duration'] = (self.meetings_df['minute_end'] - self.meetings_df['minute_start']).dt.total_seconds() / 60

        expected_duration = {'LMV': 50.0, 'MJ': 75.0}
        duration_conflicts = self.meetings_df[
            self.meetings_df.apply(lambda row: expected_duration.get(row['day']) != row['duration'], axis=1)
        ]

        self.meetings_df.drop(columns=['minute_start', 'minute_end', 'duration'], inplace=True)

    def check_overcapacity(self):
        """Ensure sections do not exceed room capacity."""
        sections_availability = self.sections_df.merge(self.rooms_df, left_on='room_id', right_on='rid', how='left')
        overcapacity_sections = sections_availability[sections_availability['capacity_x'] > sections_availability['capacity_y']]
        sids_to_drop = overcapacity_sections['sid'].tolist()
        self.sections_df = self.sections_df[~self.sections_df['sid'].isin(sids_to_drop)].reset_index(drop=True)

    def validate_sections(self):
        """Validate sections based on class, meeting, and room data."""
        valid_enrollment_df = self.sections_df.merge(self.courses_df, left_on="class_id", right_on="classid")
        valid_enrollment_df = valid_enrollment_df.merge(self.meetings_df, left_on="meeting_id", right_on="mid")
        valid_enrollment_df = valid_enrollment_df.merge(self.rooms_df, left_on="room_id", right_on="rid")
        valid_sids = valid_enrollment_df['sid'].values
        self.sections_df = self.sections_df[self.sections_df['sid'].isin(valid_sids)].reset_index(drop=True)

    def download_syllabi(self, syllabus_dir):
        """Download syllabi from provided URLs."""
        os.makedirs(syllabus_dir, exist_ok=True)
        for _, row in self.courses_df.iterrows():
            syllabus_url = row['syllabus']
            syllabus_filename = f'{syllabus_dir}/{row["classes"]["name"]}-{row["classes"]["code"]}-{row["description"]}.pdf'
            self._download_syllabus(syllabus_url, syllabus_filename)

    @staticmethod
    def _download_syllabus(url, filename):
        """Helper function to download a syllabus from a URL."""
        try:
            response = requests.get(url)
            response.raise_for_status()
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {filename}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {url}: {e}")

    def adjust_timestamps(self):
        """Adjust timestamps for meetings based on the year and semester."""
        meetings_with_year = self.meetings_df.merge(
            self.sections_df[['meeting_id', 'year', 'semester']],
            left_on='mid', right_on='meeting_id',
            how='left'
        )
        
        # Map semesters to default dates (month and day)
        semester_dates = {
            'Fall': (9, 1),    # September 1st
            'Spring': (1, 15),  # January 15th
            'V2': (6, 1),      # Assume June 1st for summer sessions
        }
        
        # Add 'month' and 'day' columns based on the semester
        meetings_with_year['month'] = meetings_with_year['semester'].map(
            lambda x: semester_dates.get(x, (1, 1))[0]
        )
        meetings_with_year['day'] = meetings_with_year['semester'].map(
            lambda x: semester_dates.get(x, (1, 1))[1]
        )
        
        # Create datetime objects for 'start' and 'end'
        meetings_with_year['start_datetime'] = meetings_with_year.apply(
            lambda row: datetime(
                year=row['year'], month=row['month'], day=row['day'],
                hour=int(row['start'].split(':')[0]),
                minute=int(row['start'].split(':')[1])
            ), axis=1
        )
        
        meetings_with_year['end_datetime'] = meetings_with_year.apply(
            lambda row: datetime(
                year=row['year'], month=row['month'], day=row['day'],
                hour=int(row['end'].split(':')[0]),
                minute=int(row['end'].split(':')[1])
            ), axis=1
        )
        
        # Update the meetings_df with new datetime columns and drop intermediate columns
        self.meetings_df['start'] = meetings_with_year['start_datetime']
        self.meetings_df['end'] = meetings_with_year['end_datetime']

    def execute_transformations(self, syllabus_dir):
        """Execute all transformations on the data."""
        self.clean_courses()
        self.resolve_section_conflicts()
        self.filter_meetings()
        self.validate_meeting_durations()
        self.check_overcapacity()
        self.validate_sections()
        self.adjust_timestamps()
        # self.download_syllabi(syllabus_dir)
        
        print("[ETL] Transformations complete.")
        
        # Return the transformed dataframes
        return self.courses_df, self.meetings_df, self.requisites_df, self.rooms_df, self.sections_df


In [20]:
# Define the directory where the data files are stored
data_dir = "/Users/romerocruzsa/Downloads/rumad2.0_data/"
syllabus_dir = "/Users/romerocruzsa/Workspace/Projects/Classes/CIIC4060/rumad-v2-app-no-pensamos-repetir-npr/syllabuses"

# Extract rumad2.0_data files
extractor = Extract(data_dir)
courses_df, meetings_df, requisites_df, rooms_df, sections_df = extractor.extract_all()

# Transform data
transformer = Transform(courses_df, meetings_df, requisites_df, rooms_df, sections_df)
courses_df, meetings_df, requisites_df, rooms_df, sections_df = transformer.execute_transformations(syllabus_dir)

[ETL] Extraction complete.
[ETL] Transformations complete.


/var/folders/pd/pytqjfv51bddy0szx15zr3_c0000gn/T/ipykernel_78413/3214981209.py:54: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.meetings_df['time_start'] = pd.to_datetime(self.meetings_df['start']).dt.time
/var/folders/pd/pytqjfv51bddy0szx15zr3_c0000gn/T/ipykernel_78413/3214981209.py:55: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.meetings_df['time_end'] = pd.to_datetime(self.meetings_df['end']).dt.time


In [21]:
display(courses_df.head(10), courses_df.info())
display(meetings_df.head(10), meetings_df.info())
display(requisites_df.head(10), requisites_df.info())
display(rooms_df.head(10), rooms_df.info())
display(sections_df.head(10), sections_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   classid      36 non-null     int64 
 1   cred         36 non-null     object
 2   description  36 non-null     object
 3   syllabus     36 non-null     object
 4   term         36 non-null     object
 5   years        36 non-null     object
 6   classes      36 non-null     object
dtypes: int64(1), object(6)
memory usage: 2.1+ KB


,classid,cred,description,syllabus,term,years,classes
0,2,4,Introduction to Computer Programming I,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '3015', 'name': 'CIIC'}"
1,3,3,Foundations of Computing,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '3075', 'name': 'CIIC'}"
2,4,3,Computer Architecture I,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '3081', 'name': 'CIIC'}"
3,5,4,Advanced Programming,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4010', 'name': 'CIIC'}"
4,6,4,Data Structures,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4020', 'name': 'CIIC'}"
5,7,3,Analysis and Design of Algorithms,https://www.uprm.edu/cse/wp-content/uploads/si...,Second Semester,Every Year,"{'code': '4025', 'name': 'CIIC'}"
6,8,3,Programming Languages,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '4030', 'name': 'CIIC'}"
7,9,4,Operating Systems,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4050', 'name': 'CIIC'}"
8,10,3,Senior Design Project I,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '4151', 'name': 'CIIC'}"
9,11,3,Database Systems,https://www.uprm.edu/cse/wp-content/uploads/si...,Second Semester,Every Year,"{'code': '4060', 'name': 'CIIC'}"


None

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 0 to 19
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   mid     16 non-null     int64         
 1   ccode   16 non-null     int64         
 2   start   16 non-null     datetime64[ns]
 3   end     16 non-null     datetime64[ns]
 4   day     16 non-null     object        
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 768.0+ bytes


,mid,ccode,start,end,day
0,1,16,2022-01-15 07:30:00,2022-01-15 08:45:00,MJ
1,2,26,2025-09-01 07:30:00,2025-09-01 08:45:00,MJ
4,5,56,2018-09-01 07:30:00,2018-09-01 08:45:00,MJ
5,6,66,2020-09-01 07:30:00,2020-09-01 08:45:00,MJ
6,7,76,2020-09-01 07:30:00,2020-09-01 08:45:00,MJ
7,8,86,2023-01-15 07:30:00,2023-01-15 08:45:00,MJ
10,11,10,2018-06-01 09:00:00,2018-06-01 10:15:00,LWV
11,12,20,2019-09-01 09:00:00,2019-09-01 10:15:00,LWV
12,13,30,2024-09-01 09:00:00,2024-09-01 10:15:00,LWV
13,14,40,2025-09-01 09:00:00,2025-09-01 10:15:00,LWV


...

,rid,building,number,capacity
0,1,Stefani,113,120
1,2,Stefani,114,25
2,3,Stefani,121,30
3,4,Stefani,215,30
4,5,Stefani,226,40
5,6,Stefani,207,30
6,7,Stefani,214,30
7,8,Stefani,322,30
8,9,Stefani,317,28
9,10,Stefani,330,30


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sid         221 non-null    int64 
 1   room_id     221 non-null    int64 
 2   meeting_id  221 non-null    int64 
 3   class_id    221 non-null    int64 
 4   semester    221 non-null    object
 5   year        221 non-null    int64 
 6   capacity    221 non-null    int64 
dtypes: int64(6), object(1)
memory usage: 12.2+ KB


,sid,room_id,meeting_id,class_id,semester,year,capacity
0,12,14,19,27,Fall,2022,28
1,13,22,1,21,Spring,2022,35
2,15,11,6,15,V2,2020,30
3,16,2,2,35,Spring,2019,18
4,17,1,17,3,Spring,2021,16
5,18,2,14,4,Fall,2020,21
6,35,18,12,2,Spring,2018,22
7,40,8,18,10,Fall,2025,24
8,47,14,16,13,Fall,2017,28
9,49,25,20,15,Spring,2023,22


None

In [28]:
courses_df

,classid,cred,description,syllabus,term,years,classes
0,2,4,Introduction to Computer Programming I,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '3015', 'name': 'CIIC'}"
1,3,3,Foundations of Computing,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '3075', 'name': 'CIIC'}"
2,4,3,Computer Architecture I,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '3081', 'name': 'CIIC'}"
3,5,4,Advanced Programming,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4010', 'name': 'CIIC'}"
4,6,4,Data Structures,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4020', 'name': 'CIIC'}"
5,7,3,Analysis and Design of Algorithms,https://www.uprm.edu/cse/wp-content/uploads/si...,Second Semester,Every Year,"{'code': '4025', 'name': 'CIIC'}"
6,8,3,Programming Languages,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '4030', 'name': 'CIIC'}"
7,9,4,Operating Systems,https://www.uprm.edu/cse/wp-content/uploads/si...,"First Semester, Second Semester",Every Year,"{'code': '4050', 'name': 'CIIC'}"
8,10,3,Senior Design Project I,https://www.uprm.edu/cse/wp-content/uploads/si...,First Semester,Every Year,"{'code': '4151', 'name': 'CIIC'}"
9,11,3,Database Systems,https://www.uprm.edu/cse/wp-content/uploads/si...,Second Semester,Every Year,"{'code': '4060', 'name': 'CIIC'}"


In [30]:
meeting_df

,mid,ccode,start,end,day
0,1,16,07:30:00,08:45:00,MJ
1,2,26,09:00:00,10:15:00,MJ
4,5,56,20:00:00,21:15:00,MJ
5,6,66,20:00:00,21:15:00,MJ
6,7,76,20:00:00,21:15:00,MJ
7,8,86,20:00:00,21:15:00,MJ
10,11,10,07:30:00,08:20:00,LWV
11,12,20,08:30:00,09:20:00,LWV
12,13,30,09:30:00,10:20:00,LWV
13,14,40,10:30:00,11:20:00,LWV


In [33]:
sections_df[sections_df["meeting_id"]==19]

,sid,room_id,meeting_id,class_id,semester,year,capacity
0,12,14,19,27,Fall,2022,28
10,51,20,19,19,V1,2020,19
29,166,11,19,12,Fall,2018,22
37,225,25,19,19,Fall,2022,38
44,276,16,19,36,Fall,2023,29
60,341,19,19,15,Spring,2021,65
77,458,25,19,28,Fall,2017,16
95,566,13,19,15,Spring,2017,23
100,584,16,19,6,Fall,2024,39
109,648,9,19,17,Fall,2017,21
